In [4]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

#, store={}, item_uuids=[]
env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 
set_url = '/search/?experiments_in_set.experiment_type=ChIP-seq&type=ExperimentSetReplicate&limit=all&status=pre-release&status=released&status=released%20to%20project'
set_url = '/search/?award.project=4DN&experiments_in_set.experiment_type=ChIP-seq&experimentset_type=replicate&lab.display_title=Feng+Yue+Lab%2C+PSU&type=ExperimentSetReplicate&status=pre-release&status=released&status=released%20to%20project'

set_to_release = [i['uuid'] for i in ff_utils.search_metadata(set_url, my_auth)]
store, uuids = record_object_es(set_to_release, my_auth, schema_name, add_pc_wfr=True, store_frame='object')
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print((time2-time1))

225
225
5.8640501499176025


In [5]:
for i in store:
    print(i, len(store[i]))

workflow 6
file_reference 3
software 12
file_fastq 20
file_processed 23
workflow_run_awsem 44
biosource 1
vendor 4
protocol 2
quality_metric_chipseq 11
quality_metric_fastqc 16
target 4
ontology_term 10
experiment_seq 10
biosample_cell_culture 2
individual_human 1
lab 9
organism 1
file_format 8
user 17
static_section 1
antibody 4
ontology 2
biosample 2
experiment_set_replicate 4
award 8


In [6]:
# TODO
# Check audits

# do you want to check for duplicate/problematic runs on files?
# it will take some time
check_wfrs = True
delete_problematic = False

# check expsets
print('EXPSET CHECK')
for a_set in store['experiment_set_replicate']:
    if not a_set.get('completed_processes'):
        print(a_set['accession'], 'missing processing tag', a_set['description'][:50])

# check exps 
print('\nEXP CHECK')
# check for experiment numbers
exp_names = ['experiment_hi_c', 'experiment_mic', 'experiment_seq']
all_exps_on_sets = [a for i in store['experiment_set_replicate'] for a in i['experiments_in_set']]
all_exps = [a['uuid'] for i in store.keys() for a in store[i] if i in exp_names]
if len(all_exps_on_sets) != len(all_exps):
    print('Number of experiments is not same as experiments associated with sets')
    print('# of exps: {}. # of exps on sets: {}'.format(len(store['experiment_seq']), len(all_exps_on_sets)))
        
print('\nFILE FASTQ CHECK')
for a_file in store['file_fastq']:
    if a_file['status'] in ['uploading', 'upload failed']:
        print(a_file['accession'], 'not yet uploaded')
    if not a_file.get('quality_metric'):
        print(a_file['accession'], 'missing fastqc')
    if not a_file.get('content_md5sum') or not a_file.get('md5sum'):
        print(a_file['accession'], 'problems with md5')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)

  

# check processed files
print('\nFILE PROCESSED CHECK')
for a_file in store['file_processed']:
    if a_file['file_format'] == '/file-formats/pairs/':
        if not a_file.get('quality_metric'):
            print(a_file['accession'], 'missing Pairsqc')
    if not a_file.get('source_experiments'):
        print(a_file['accession'], 'missing source experiments')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)   

# check wfrs
print('\nWFR CHECK')
for wfr in store['workflow_run_awsem']:
    if wfr['run_status'] != 'complete':
        print('problematic wfr', wfr['uuid'], wfr['run_status'])
        
# check for weird status
print('\nREPORT NUMBERS AND CHECK STATUS')
for i in store:
    print(i, len(store[i]))
    weird = [[i,x['uuid'],x['status']] for x in store[i] if x['status'] in ['deleted', 'replaced', 'archived', 'revoked']]
    if weird:
        for case in weird:
            print(case)
        print()


EXPSET CHECK

EXP CHECK
Number of experiments is not same as experiments associated with sets
# of exps: 10. # of exps on sets: 8

FILE FASTQ CHECK
4DNFI4IKEMGX not yet uploaded
4DNFI4IKEMGX missing fastqc
4DNFI4IKEMGX problems with md5
4DNFILF1UY2Y not yet uploaded
4DNFILF1UY2Y missing fastqc
4DNFILF1UY2Y problems with md5
4DNFI88R8KZ7 not yet uploaded
4DNFI88R8KZ7 missing fastqc
4DNFI88R8KZ7 problems with md5
4DNFI5LI1ZLM not yet uploaded
4DNFI5LI1ZLM missing fastqc
4DNFI5LI1ZLM problems with md5

FILE PROCESSED CHECK
encode-chipseq-postaln 0.2.5 old style or dub 8016ddc3-d361-45ea-8a90-98a4373a3f28 4DNFI6LXZHOQ
encode-chipseq-postaln 0.2.5 old style or dub 9d36f764-df90-4663-a218-473e2bf3f66f 4DNFI6LXZHOQ
encode-chipseq-postaln 0.2.5 old style or dub 8016ddc3-d361-45ea-8a90-98a4373a3f28 4DNFIIHW2GRI
encode-chipseq-postaln 0.2.5 old style or dub 9d36f764-df90-4663-a218-473e2bf3f66f 4DNFIIHW2GRI

WFR CHECK

REPORT NUMBERS AND CHECK STATUS
workflow 6
file_reference 3
software 12
file_f

In [ ]:
# Check status
change_status = 'released'

common_types = ['lab', 'user', 'award','ontology', 'ontology_term', 
                'file_format', 'software', 'workflow', 'static_section',
                'organism', 'vendor', 'file_reference', 'individual_human', 
                'enzyme', 'biosource', 'file_format']

print('COMMON ITEMS')
for a_type in store:
    if a_type in common_types:
        for raw_data in store[a_type]:
            if raw_data['status'] not in ['released', 'current']:
                print('COMMON ITEM NOT RELEASED', a_type,raw_data['status'], raw_data['uuid'])
print('---------------------------------\n')

counter = 0
for a_type in store:
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] != change_status:
            print(a_type, raw_data['uuid'], raw_data['status'])



In [ ]:
# Release the items
action = True
change_status = 'released'

counter = 0
for a_type in store:
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print('deleted data', a_type ,raw_data['uuid'])
            continue        
        if raw_data['status'] not in ['current', 'released']:    
            counter +=1
            print(a_type, raw_data['status'], raw_data['uuid'])
            patch_data = {'status': change_status}
            if action:
                if change_status == 'released':
                    if a_type in ['publication', 'user', 'award', 'vendor', 'organism', ]:
                        ff_utils.patch_metadata({'status': 'current'}, obj_id=raw_data['uuid'] ,key=my_auth)
                    else:
                        ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
                else:
                    ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
                
        else:
            print('ALREADY RELEASED/TO PROJECT', a_type, raw_data['status'], raw_data['uuid'])
            pass
            
print(counter)